In [30]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:

class AnimalDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.samples = self._make_dataset()

    def _make_dataset(self):
        samples = []
        for class_name in self.classes:
            class_dir = os.path.join(self.root_dir, class_name)
            if not os.path.isdir(class_dir):
                continue

            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                if os.path.isfile(img_path):
                    samples.append((img_path, self.class_to_idx[class_name]))
        return samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

Створення трансформацій, вирівнювання розмірностей, ініціалізація

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def create_dataloaders(dataset_path, batch_size=32, val_split=0.2):
    full_dataset = AnimalDataset(root_dir=dataset_path, transform=train_transform)

    val_size = int(len(full_dataset) * val_split)
    train_size = len(full_dataset) - val_size

    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    val_dataset.dataset.transform = val_transform

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, val_loader, full_dataset.classes

Тринувальна, валідаційна частини:

In [33]:
dataset_path = 'raw-img'
train_loader, val_loader, classes = create_dataloaders(dataset_path, batch_size=32, val_split=0.2)

print(f"Розмір тренувального набору: {len(train_loader.dataset)} зображень")
print(f"Розмір валідаційного набору: {len(val_loader.dataset)} зображень")
print(f"Кількість батчів у тренувальному завантажувачі: {len(train_loader)}")
print(f"Кількість батчів у валідаційному завантажувачі: {len(val_loader)}")

Розмір тренувального набору: 20944 зображень
Розмір валідаційного набору: 5235 зображень
Кількість батчів у тренувальному завантажувачі: 655
Кількість батчів у валідаційному завантажувачі: 164
